In [12]:
import pandas as pd
import os

In [23]:
directory = os.fsencode("/workspace/gideon_data_2023-07-14")
csv_sheets = {}
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        csv_sheets[filename.split("__")[0]] = pd.read_csv("/workspace/gideon_data_2023-07-14/" + filename)

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (61,62,67,88,90,97,98) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:

csv_sheets.keys()

dict_keys(['_D_BIOGRAFIEN', '_K_PUBLIKATIONSARTEN', '_K_LAENDER', '_K_PUBLIKATIONSKATEGORIEN', '_MITARBEITER', '_BIOGRAFIEN_PUBLIKATIONEN', '_BIOGRAFIE_LITERATUR', '_K_PUBLIKATIONSSTAUS', '_K_BERUFSGRUPPEN', '_K_BERUFSUNTERGRUPPEN'])

In [4]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["DJANGO_SETTINGS_MODULE"] = "apis.settings.dev_remote_db"
import django
django.setup()                                                                                                          

inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes
inspecting classes


In [19]:
from oebl_research_backend.models import *
import pandas as pd
import numpy as np
import re

In [7]:
IRSPerson.objects.all().count()

79

In [21]:

map_user_gideon = {
    776923: 13,
    776922: 12,
    2236853: 8,
    776924: 16,
    776926: 17,
    2083043: 9,
    1870444: 14
}

d_mappings = {
    "gender": {"1": "männlich",
                "2": "weiblich"
                }
}

boolean_vals = ["IST_ADELIG", ".*_OK", "RELIGIONSWECHSEL", "BIO", "PUBLIZIERT", "RELIGIONSWECHSEL"]
pat_booleans = []
for pat in boolean_vals:
    pat_booleans.append(re.compile(pat))

map_csv_irs = {
    "ADELSPRAEDIKAT": "nobleTitle"
}

exclude_columns = [".*_2", "DUMMY_.*", "TEST", "WERK_.*", "WEB_.*", "K_.*", "SUCH_.*", ".*_OK", 
"NACHNAME", "VORNAME", "KK", "WF", "DOMAIN", "ERRATA", "ADDENDA", "AKTUELL", "PROJECT", "VERWEIS", "WEITERE_NAMENSFORMEN",
"MODIFIZIERT_VON", "DUMMY", "KOE", "WEB", "STATUS_ID", "STERBETAG", "STERBEMONAT", "STERBEJAHR", "GBEURTSTAG", "GEBURTSTAGMONAT",
"GEBURTSTAGJAHR", "PNDID"
]
ex_patterns = []
for pat in exclude_columns:
    ex_patterns.append(re.compile(pat))

In [20]:

def get_value(value, col):
    for pat in pat_booleans:
        if pat.fullmatch(col):
            if isinstance(value, str):
                if value.strip() == "n":
                    return True
                elif value.strip() == "f":
                    return False
                else:
                    return None
            elif isinstance(value, float):
                if np.isnan(value):
                    return None
                elif value == -1.0:
                    return True
                elif value == 0.0:
                    return False
                else:
                    return None
    if isinstance(value, float):
        if np.isnan(value):
            return None
        else:
            return value
    elif isinstance(value, str):
        if value == "?":
            return None
        return value

In [22]:
def test_col_user(col):
    for pat in ex_patterns:
        if pat.fullmatch(col):
            return False
    return True

In [53]:
lists_ent = {}
for idx, row in csv_sheets["_D_BIOGRAFIEN"].iterrows():
    # if np.isnan(row["STERBEJAHR"]):
    #     continue
    # if int(row["STERBEJAHR"]) < 1951:
    #     continue
    # if idx < 31:
    #     continue
    pers = {
        "name": get_value(row["NACHNAME"], "NACHNAME"),
        "first_name": get_value(row["VORNAME"], "VORNAME"),
    }
    if not np.isnan(row["GEBURTSJAHR"]):
        pers["date_of_birth"] = f"{int(row['GEBURTSJAHR'])}"
        if not np.isnan(row["GEBURTSTAGMONAT"]):
            pers["date_of_birth"] += f'-{int(row["GEBURTSTAGMONAT"])}'.strip()
            if not np.isnan(row["GBEURTSTAG"]):
                pers["date_of_birth"] += f'-{int(row["GBEURTSTAG"])}'.strip()
            else:
                pers["date_of_birth"] += '-01'
        else:
            pers["date_of_birth"] += '-01-01'
    if not np.isnan(row["STERBEJAHR"]):
        pers["date_of_death"] = f"{int(row['STERBEJAHR'])}"
        if not np.isnan(row["STERBETAGMONAT"]):
            pers["date_of_death"] += f'-{int(row["STERBETAGMONAT"])}'.strip()
            if not np.isnan(row["STERBETAG"]):
                pers["date_of_death"] += f'-{int(row["STERBETAG"])}'.strip()
            else:
                pers["date_of_death"] += '-01'
        else:
            pers["date_of_death"] += '-01-01'
    if row["K_GESCHLECHT_ID"] == 1:
        pers["gender"] = "männlich" 
    elif row["K_GESCHLECHT_ID"] == 2:
        pers["gender"] = "weiblich"
    else:
        pers["gender"] = "unbekannt"
    if not np.isnan(row["K_BERUFSGRUPPEN_ID"]):
        r_bg = csv_sheets["_K_BERUFSGRUPPEN"].loc[(csv_sheets["_K_BERUFSGRUPPEN"]["ID"] == row["K_BERUFSGRUPPEN_ID"])]
        if r_bg.shape[0] == 1:
            bg, created = ProfessionGroup.objects.get_or_create(name=r_bg.iloc[0]["TITEL"])
            pers["profession_group"] = bg
        elif r_bg.shape[0] > 1:
            print(f"several profession groups, row {idx}")
        else:
            print(f"no profession group specified, row {idx}")
    if isinstance(row["BERUF"], str):
        pers["profession_detail"] = get_value(row["BERUF"], "BERUF")
    lit_gideon = []
    for idx_lit, row_lit in csv_sheets["_BIOGRAFIE_LITERATUR"].loc[(csv_sheets["_BIOGRAFIE_LITERATUR"]["BIOGRAFIE_ID"] == row["ID"])].iterrows():
        lit_gideon.append({
            "id": row_lit["ID"],
            "value": row_lit["ZITAT"]
        })
    if len(lit_gideon) > 0:
        pers["gideon_legacy_literature"] = lit_gideon
    #pers["alternative_names"] = [{"first_name":"test1", "last_name": "test2"}]
    if isinstance(row["LEBENSLAUF"], str):
        if len(row["LEBENSLAUF"]) > 0:
            pers["bio_note"] = row["LEBENSLAUF"]
    if isinstance(row["WEITERE_NAMENSFORMEN"], str):
        if len(row["WEITERE_NAMENSFORMEN"]) > 0:
            pers["alternative_names"] = [{
                "first_name": "-",
                "last_name": row["WEITERE_NAMENSFORMEN"]
            }]
    pers["uris"] = []
    if isinstance(row["PNDID"], str):
        pers["uris"] = [f"https://d-nb.info/gnd/{row['PNDID']}", ]
    try:
        i1 = IRSPerson.objects.create(**pers)
    except Exception as e:
        print(f"error in row {idx}: {e}")
        continue
    list_ent_dict = {
        "person": i1, 
        "source_id": int(row["ID"]),
        "columns_scrape": {"obv": [], "wikipedia": [], "wikidata": []},
        "scrape": {"obv": [], "wikipedia": [], "wikidata": []},
        "columns_user": {"selected": False}
        }
    pub_entry = csv_sheets["_BIOGRAFIEN_PUBLIKATIONEN"].loc[(csv_sheets["_BIOGRAFIEN_PUBLIKATIONEN"]["BIOGRAFIE_ID"] == row["ID"])]
    if len(pub_entry) > 0:
        print(pub_entry)
        print(pub_entry["PUB_STATUS_ID"].iloc[0])
        list_ent_dict["columns_user"]["Gideon | Status"] = csv_sheets["_K_PUBLIKATIONSSTAUS"].loc[(csv_sheets["_K_PUBLIKATIONSSTAUS"]["ID"] == pub_entry["PUB_STATUS_ID"].iloc[0])]["TITEL"].iloc[0]
        list_ent_dict["columns_user"]["Gideon | Kategorie"] = csv_sheets["_K_PUBLIKATIONSKATEGORIEN"].loc[(csv_sheets["_K_PUBLIKATIONSKATEGORIEN"]["ID"] == pub_entry["K_PUBLIKATIONSKATEGORIEN_ID"].iloc[0])]["TITEL"].iloc[0]
        list_ent_dict["columns_user"]["Gideon | Art"] = csv_sheets["_K_PUBLIKATIONSARTEN"].loc[(csv_sheets["_K_PUBLIKATIONSARTEN"]["ID"] == pub_entry["PUBLIKATIONS_ART"].iloc[0])]["TITEL"].iloc[0]
        list_ent_dict["columns_user"]["Gideon | Band"] = pub_entry["BAND"].iloc[0] if not np.isnan(pub_entry["BAND"].iloc[0]) else "-"
        list_ent_dict["columns_user"]["Gideon | Lieferung"] = pub_entry["LIEFERUNG"].iloc[0] if not np.isnan(pub_entry["LIEFERUNG"].iloc[0]) else "-"
        list_ent_dict["columns_user"]["Gideon | Publikationsjahr"] = pub_entry["PUB_JJJJ"].iloc[0] if not np.isnan(pub_entry["PUB_JJJJ"].iloc[0]) else "-"
        if not np.isnan(pub_entry["FACHREDAKTEUR_ID"].iloc[0]):
            editor = csv_sheets["_MITARBEITER"].loc[(csv_sheets["_MITARBEITER"]["ID"] == pub_entry["FACHREDAKTEUR_ID"].iloc[0])]
            if editor.shape[0] == 1:
                editor_name = editor.iloc[0]["NAME"]
                if editor_name not in lists_ent.keys():
                    d_list = {"title": f"Gideon {editor_name}"}
                    if int(row["USERID"]) in map_user_gideon.keys():
                        d_list["editor_id"] = map_user_gideon[int(row["USERID"])]
                    list_new, created = List.objects.get_or_create(**d_list)
                    lists_ent[editor_name] = list_new
                list_ent_dict["list"] = lists_ent[editor_name]
    for col in csv_sheets["_D_BIOGRAFIEN"].columns:
        if not test_col_user(col):
            continue
        if isinstance(row[col], float):
            if np.isnan(row[col]):
                continue
        col_name = col
        if col in map_csv_irs.keys():
            col_name = map_csv_irs[col]
        list_ent_dict["columns_user"][col_name] = get_value(row[col], col)

    ListEntry.objects.create(**list_ent_dict)


KeyboardInterrupt: 

In [52]:
IRSPerson.objects.all().delete()
ListEntry.objects.all().delete()
List.objects.all().delete()

(14, {'oebl_research_backend.List': 14})

In [51]:
IRSPerson.objects.all().count()

12057